In [1]:
import logging
logger = logging.getLogger()
for handler in logger.handlers:
    handler.level = logging.WARNING
handler = logging.FileHandler(filename='heat.log', mode='w')
handler.level = logging.DEBUG
logger.level = logging.DEBUG
logger.addHandler(handler)

In [2]:
%matplotlib inline
from autograd import numpy as np
import numpy.testing
np.testing = numpy.testing
import scipy.spatial
import bayesian_pdes as bpdes
import six
import sympy as sp
import matplotlib.pyplot as plt
import autograd

In [3]:
insulated = False

In [4]:
from tempfile import NamedTemporaryFile

VIDEO_TAG = """<video controls>
 <source src="data:video/x-m4v;base64,{0}" type="video/mp4">
 Your browser does not support the video tag.
</video>"""

def anim_to_html(anim):
    if not hasattr(anim, '_encoded_video'):
        with NamedTemporaryFile(suffix='.mp4') as f:
            anim.save(f.name, fps=20, extra_args=['-vcodec', 'libx264'])
            video = open(f.name, "rb").read()
        anim._encoded_video = video.encode("base64")
    
    return VIDEO_TAG.format(anim._encoded_video)
from matplotlib import animation
animation.Animation._repr_html_ = anim_to_html

# Forward Problem

In [52]:
x, y, t, t_bar = sp.symbols('x, y, t, t_bar')
ls_x, ls_t, alpha = sp.symbols('ls_x, ls_t, alpha')
d_t = sp.sqrt((t-t_bar)**2)
k_x = sp.exp(-(x - y)**2 / (2.*ls_x**2))

#k_t = (1 + sp.sqrt(5.)*d_t / ls_t + 5.*d_t**2 / (3.*ls_t**2)) * sp.exp(-sp.sqrt(5.)/ls_t*d_t)
#k_t = sp.exp(-(t-t_bar)**2 / (2.*ls_t**2))
k_t = (1-d_t/ls_t)**5 * (8*d_t**2/ls_t**2 + 5*d_t / ls_t + 1)
k = k_x * k_t


def T(k):
    return k.diff(t) - alpha * k.diff(x,x)
def T_bar(k):
    return k.diff(t_bar) - alpha * k.diff(y, y)
def neumann(k):
    return k.diff(x)
def neumann_bar(k):
    return k.diff(y)
def B(k):
    return k
def B_bar(k):
    return k

all = [T, neumann, B]
all_bar = [T_bar, neumann_bar, B_bar]
ops = [T, B]
ops_bar = [T_bar, B_bar]

In [53]:
op_system_A = bpdes.operator_compilation.compile_sympy(
    all, 
    all_bar, 
    k, 
    [[x, t], [y, t_bar], [ls_x, ls_t, alpha]],
    mode='cython'
)

In [54]:
op_system_B = bpdes.operator_compilation.sympy_gram.compile_sympy(
    all, 
    all_bar, 
    k, 
    [[x, t], [y, t_bar], [ls_x, ls_t, alpha]], 
    limits=[(t, t_bar)],
    supports=[sp.Abs(t-t_bar) < ls_t],
    clean=False
)

In [55]:
def g(x):
    return np.zeros((x.shape[0], 1))
def b(x):
    return 0.5*np.ones((x.shape[0], 1))
def b0(x):
    return np.zeros((x.shape[0], 1))

In [56]:
def u_0_insulated(x):
    return 1-4*(x-0.5)**2 
def u_0(x):
    return np.zeros((x.shape[0], 1))

In [78]:
L = 1.
Tend = 1.0
dx = 0.05
dt = 0.02
times = np.arange(0, Tend+dt, dt)
x_pts = np.arange(0,L+dx,dx)[:,None]
interior = x_pts[1:-1]
bdy = x_pts[[0, -1]]

fun_args = np.array([1.5*dx, 4*dt, 1.0])

test = np.linspace(0,L,51)[:,None]
init_points = np.column_stack([x_pts, times[0]*np.ones_like(x_pts)])
init_obs = u_0(x_pts)
ics = [(init_points, init_obs)]
ics_ins = [(init_points, u_0_insulated(x_pts))]

In [79]:
op_system = op_system_B

In [80]:
def obs_function(t):
    interior_t = np.column_stack([interior, t*np.ones_like(interior)])
    bdy_t = np.column_stack([bdy, t*np.ones_like(bdy)])
    
    return [(interior_t, g(interior_t)), (bdy_t, b(bdy_t))]

def obs_function_0(t):
    interior_t = np.column_stack([interior, t*np.ones_like(interior)])
    bdy_t = np.column_stack([bdy, t*np.ones_like(bdy)])
    
    return [(interior_t, g(interior_t)), (bdy_t, b0(bdy_t))]

def test_function(t):
    test_t = np.column_stack([test, t*np.ones_like(test)])
    return test_t

In [81]:
all_ts, all_xs = np.meshgrid(times[0], x_pts, indexing='ij')
all_initial = np.column_stack([all_xs.ravel(), all_ts.ravel()])
all_ts, all_xs = np.meshgrid(times[1:], interior, indexing='ij')
all_interior = np.column_stack([all_xs.ravel(), all_ts.ravel()])
all_ts, all_xs = np.meshgrid(times[1:], bdy, indexing='ij')
all_boundary = np.column_stack([all_xs.ravel(), all_ts.ravel()])
all_obs = [(all_initial, u_0(all_initial)), 
           (all_design_points, g(all_design_points)), 
           (all_boundary, b(all_boundary))]

In [82]:
posterior = bpdes.collocate([()] + ops, [()] + ops_bar, all_obs, op_system_B, fun_args)

In [83]:
means, covs = [], []
for time in times:
    #print time
    test_points = test_function(time)
    mu, cov = posterior(test_points)
    means.append(mu)
    covs.append(cov)

In [84]:
means = np.column_stack(means)

In [85]:
# First set up the figure, the axis, and the plot element we want to animate
ax = plt.gca()
ax.set_xlim(x_pts.min(), x_pts.max())
ax.set_ylim(-0.1, 1.1)

fig = plt.gcf()
line, = ax.plot([], [])
ub, = ax.plot([], [], linestyle='--', c='black')
lb, = ax.plot([], [], linestyle='--', c='black')
# initialization function: plot the background of each frame
def init():
    return []

# animation function.  This is called sequentially
def animate(i):
    cov = covs[i]
    mean = means[:,i]
    line.set_data(test, mean)
    ub.set_data(test, mean+np.diag(cov))
    lb.set_data(test, mean-np.diag(cov))
    ax.set_title('t={}'.format(times[i]))
    return line,

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=means.shape[1], interval=20, blit=False)
plt.close()
anim

In [17]:
alpha_euler = 1.0

In [18]:
euler_points = np.linspace(0,L, 101)
dx = euler_points[1] - euler_points[0]
times_euler = np.arange(0, 1, 1e-5)
dt_euler = times_euler[1] - times_euler[0]
print alpha_euler * dt_euler / dx**2
initial = u_0(euler_points).ravel()

us = [initial]
u_curr = initial
for t in times_euler[1:]:
    u_next_interior = u_curr[1:-1] + alpha_euler * dt_euler / dx**2 * (u_curr[2:] - 2*u_curr[1:-1] + u_curr[:-2])
    u_next = np.concatenate([[1], u_next_interior, [1]])
    us.append(u_next)
    u_curr = u_next

0.1


In [19]:
# First set up the figure, the axis, and the plot element we want to animate
ax = plt.gca()
ax.set_xlim(x_pts.min(), x_pts.max())
ax.set_ylim(0, 1.1)

skip = 1000
u_skip = us[::skip]
times_euler_skip = times_euler[::skip]
fig = plt.gcf()
line, = ax.plot([], [])
# initialization function: plot the background of each frame
def init():
    return []

# animation function.  This is called sequentially
def animate(i):
    fn = u_skip[i]
    line.set_data(euler_points, fn)
    ax.set_title('t={}'.format(times_euler_skip[i]))
    return line,

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=len(u_skip), interval=20, blit=False)
plt.close()
anim

In [20]:
dt = sp.symbols('dt')
def T_backward(k):
    return k - alpha * dt * k.diff(x,x)
def T_bar_backward(k):
    return k - alpha * dt * k.diff(y,y)

In [21]:
all_backward = [T_backward, neumann, B]
all_bar_backward = [T_bar_backward, neumann_bar, B_bar]
op_system_backward = bpdes.operator_compilation.sympy_gram.compile_sympy(
    all_backward,
    all_bar_backward,
    k_x,
    [[x], [y], [ls_x, alpha, dt]]
)

In [22]:
obs = [
    (interior, np.zeros((interior.shape[0], 1))),
    (bdy, np.ones((bdy.shape[0], 1)))
]

In [23]:
# need noisy observations in collocate!!!
for time in times:
    posterior = bpdes.collocate()

TypeError: collocate() takes at least 4 arguments (0 given)